# Fine-tuning GPT-4o to Write LinkedIn Posts (in my style)
## ABB #2 - Session 5

Code authored by: Shaw Talebi

### imports

In [1]:
import os
import csv
import json
import random

from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# import sk from .env file
load_dotenv()
my_sk = os.getenv("OPENAI_API_KEY")

# connect to openai API
client = OpenAI(api_key=my_sk)

### Read data

In [3]:
# load csv of YouTube comments
idea_list = []
copy_list = []
media_list = []

with open('data/LI_posts.csv', mode ='r') as file:
    file = csv.reader(file)
    
    # read file line by line
    for line in file:
        # skip first line
        if line[0]=='Idea':
            continue
            
        # append comments and responses to respective lists
        idea_list.append(line[0])
        copy_list.append(line[1])
        media_list.append(line[2])

In [4]:
print(len(idea_list))
print(len(copy_list))
print(len(media_list))

50
50
50


### Create training examples

In [5]:
# construct training examples
example_list = []

system_prompt = "LinkedIn Post Writer for Shaw Talebi, AI educator and entrepreneur"

prompt_template = lambda idea_string : f"""Write a LinkedIn post based on the following idea:
{idea_string}

Include:
- A compelling opening line that hooks the reader
- Copy that expands upon the idea in valuable way
- A call to action or share relevant content

Output:
"""

for i in range(len(idea_list)):    
    system_dict = {"role": "system", "content": system_prompt}
    user_dict = {"role": "user", "content": prompt_template(idea_list[i])}
    assistant_dict = {"role": "assistant", "content": copy_list[i] + "\n\n--\nMedia: " + media_list[i]}
    
    messages_list = [system_dict, user_dict, assistant_dict]
    
    example_list.append({"messages": messages_list})

In [6]:
print(example_list[0]['messages'][0]['content'])
print(example_list[0]['messages'][1]['content'])
print(example_list[0]['messages'][2]['content'])

LinkedIn Post Writer for Shaw Talebi, AI educator and entrepreneur
Write a LinkedIn post based on the following idea:
3 types of AI Tik Tok

Include:
- A compelling opening line that hooks the reader
- Copy that expands upon the idea in valuable way
- A call to action or share relevant content

Output:

A problem with AI today is that it means different things to different people. 

This framework from Andrej Karpathy helped give me much more clarity 👇 

Software 1.0 = Rule-based software systems. Humans program computers to solve problems step-by-step. 

Software 2.0 = Computers program themselves by seeing examples (i.e. machine learning) 

Software 3.0 = Repurposing general-purpose ML models for specific use cases (i.e. GenAI + Foundation Models) 

But… what’s Software 4.0 going to be? 🤔

--
Media: Video


In [7]:
len(example_list)

50

### Create train/validation split

In [8]:
# randomly pick out validation examples
num_examples = 10
validation_index_list = random.sample(range(0, len(example_list)-1), num_examples)
validation_data_list = [example_list[index] for index in validation_index_list]

for example in validation_data_list:
    example_list.remove(example)

In [9]:
print(len(example_list))
print(len(validation_data_list))

40
10


In [10]:
# write examples to file
with open('data/train-data.jsonl', 'w') as train_file:
    for example in example_list:
        json.dump(example, train_file)
        train_file.write('\n')

with open('data/valid-data.jsonl', 'w') as valid_file:
    for example in validation_data_list:
        json.dump(example, valid_file)
        valid_file.write('\n')

### Upload data to OpenAI

In [11]:
train_file = client.files.create(
  file = open("data/train-data.jsonl", "rb"),
  purpose = "fine-tune"
)

valid_file = client.files.create(
  file = open("data/valid-data.jsonl", "rb"),
  purpose = "fine-tune"
)

### Fine-tune model

In [24]:
client.fine_tuning.jobs.create(
    training_file = train_file.id,
    validation_file = valid_file.id,
    suffix = "LI-post-writer",
    model = "gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-7DMn3tEj85tB7M9Ndp7P8a5J', created_at=1741302497, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='ft:gpt-4o-mini-2024-07-18:shawhin-talebi-ventures-llc:li-post-writer:B0uwdtNl', object='fine_tuning.job', organization_id='org-KjWERyZ9WLUqIdrdMeJh4zC0', result_files=[], seed=1090180453, status='validating_files', trained_tokens=None, training_file='file-CB3scQJZCELES9HWsRc1PX', validation_file='file-GjUq4kiKjV9wyur3caz6Wi', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix='LI-post-writer', metadata=None)

### Evaluate fine-tuned model

In [13]:
def generate_post(system_prompt, model_name, idea):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_template(idea)}
        ],
        temperature=1,
    )
    return response.choices[0].message.content

In [15]:
idea = "How I’d Learn AI in 2025 (If I Knew Nothing). Step 1: Use ChatGPT (or the like).Step 2: Install Python.Step 3: Build an Automation (Beginner).Step 4: Build an ML Project (Intermediate).Step 5: Build a Real-world Project (Advanced)Promote blog."

In [16]:
# GPT-4o (no fine-tuning)
model_name = "gpt-4o"
system_prompt_long = "You are an AI assistant helping Shaw Talebi, an AI educator and entrepreneur, craft LinkedIn posts. Your goal is to generate posts \
that reflect Shaw Talebi's voice: authoritative yet approachable, insightful yet concise. Shaw Talebi's posts aim to educate and inspire professionals \
in the tech and AI space. Focus on providing value, discussing new trends, or offering actionable advice, while keeping the tone professional but \
conversational. The target audience includes entrepreneurs, tech professionals, and decision-makers in AI and data science. Always ensure the post is \
relevant, engaging, and on-brand for Shaw Talebi's public persona."

# print(system_prompt_long, "\n--")
print(generate_post(system_prompt_long, model_name, idea))

Ever wondered how to dive into AI from scratch in 2025? 🤔 Here's your roadmap to mastering the future, step by step.

1. **Use AI's Own Tools**: Begin with ChatGPT or a similar conversational AI. They’re not just for questions—think of them as your code buddy, guiding you through concepts and debugging alongside you.

2. **Install Python**: This isn't just about installing software; you're setting up the foundation. Python's versatility makes it indispensable in AI.

3. **Build an Automation (Beginner)**: Start small. Automate a simple task. It's hands-on experience that makes all the difference. It’s your gateway into understanding logic and problem-solving.

4. **Build an ML Project (Intermediate)**: Time to elevate your skills with machine learning. Dive into a project like sentiment analysis or a recommendation system. This is where you'll see algorithms in action.

5. **Build a Real-World Project (Advanced)**: Finally, tackle a problem that resonates with you or your community. Wh

In [27]:
# GPT-4o-mini (fine-tuned)
model_name = "ft:gpt-4o-mini-2024-07-18:shawhin-talebi-ventures-llc:li-post-writer:B8EjPxp8"

# print(system_prompt, "\n--")
print(generate_post(system_prompt, model_name, idea))

In 2025, this is how I’d learn AI… if I knew nothing 🤖 

Step 1: Chat with ChatGPT (or the like) 

Step 2: Install Python 

Step 3: Build an Automation (Beginner) 

Step 4: Build an ML Project (Intermediate) 

Step 5: Build a Real-world Project (Advanced) 

I expanded this framework in a blog post https://lnkd.in/gubGdcNn 

Thanks to Towards Data Science for publishing this article (my 12th TDS article) 

#AI #machinelearning #python

--
Media: Blog link


In [18]:
# # delete files (after fine-tuning is done)
# client.files.delete(train_file.id)
# client.files.delete(valid_file.id)